<a href="https://colab.research.google.com/github/scancer-org/data-prep-and-model-train/blob/main/Wandb%20%2B%20PCAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### TODO:
# Load data - this is the major phase here
# Sample out of the data - like 100 examples for train, 20 example for test, same distribution of classes
# Have some stats on it
# Create (in PyTorch) a base model
# Train
# Eval
# Test

In [ ]:
%%capture
!pip install -qqq wandb pytorch-lightning

In [ ]:
# Test labels (from original, which I'll split for train / test)
import h5py
import numpy as np
import torch
import wandb
import os
import pandas as pd
import PIL.Image
import matplotlib.pyplot as plt
import shutil
import time
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl

from torch.utils.data import DataLoader, Dataset
from google.colab import drive
from torch.utils import data
from os import listdir
from pathlib import Path
from PIL import Image
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, datasets

# anyeone interested should have this path configured already, dataset can be downloaded from GitHub / Drive (PCAM)
# Batch Size = 128
CHECKPOINT_DIR = 'checkpoint'
drive_base_path = 'gdrive/MyDrive/pcamv1/'
BATCH_SIZE = 16

In [ ]:
pl.seed_everything(hash("setting random seeds") % 2**32 -1)

# wandblogger <> from pytorch ligthning!
from pytorch_lightning.loggers import WandbLogger

wandb.login()

Global seed set to 3551501822


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
wandb.init(project="pcam-pytorch-training")

wandb: Currently logged in as: daniel8hen (use `wandb login --relogin` to force relogin)


In [ ]:
drive.mount('/content/gdrive/')
!ls gdrive/MyDrive/pcamv1

Mounted at /content/gdrive/
camelyonpatch_level_2_split_test_meta.csv
camelyonpatch_level_2_split_test_x.h5
camelyonpatch_level_2_split_test_y.h5
camelyonpatch_level_2_split_train_mask.h5
camelyonpatch_level_2_split_train_meta.csv
camelyonpatch_level_2_split_train_x.h5
camelyonpatch_level_2_split_train_y.h5
camelyonpatch_level_2_split_valid_meta.csv
camelyonpatch_level_2_split_valid_x.h5
camelyonpatch_level_2_split_valid_y.h5


In [ ]:
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_valid_x.h5.gz
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_test_y.h5.gz
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_train_y.h5.gz
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_valid_y.h5.gz
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_test_x.h5.gz
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_train_mask.h5.gz
# !gzip -d gdrive/MyDrive/pcamv1/camelyonpatch_level_2_split_train_x.h5.gz

In [ ]:
# class H5_PCAM_DataModule(pl.LightningDataModule):
  
#   def __init__(self, path, batch_size=BATCH_SIZE):
#     super().__init__()
#     self.path = path
#     self.dataset_x = None
#     self.dataset_y = None

#     self.batch_size = batch_size
    
#   def prepare_data(self):
#     # train
#     file_x_train = h5py.File(self.path + '_x.h5', 'r')
#     file_y_train = h5py.File(self.path + '_y.h5', 'r')
#     # validation
#     file_x_val = h5py.File(self.path + '_x.h5', 'r')
#     file_y_val = h5py.File(self.path + '_y.h5', 'r')
#     # test
#     file_x_test = h5py.File(self.path + '_x.h5', 'r')
#     file_y_test = h5py.File(self.path + '_y.h5', 'r')

#   def setup(self, stage=None):
#     train_transforms = transforms.Compose([
#           transforms.RandomVerticalFlip(),
#           transforms.RandomHorizontalFlip(),
#           transforms.ToTensor(),
#           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#     test_transforms = transforms.Compose([
#           transforms.ToTensor(),
#           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#     ])

#     self.train = (file_x_train, file_y_train)
#     self.val = (file_x_val, file_y_val)
#     self.test = (file_x_test, file_y_test)

#   def train_dataloader(self):
#     return DataLoader(self.train, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)


#   def val_dataloader(self):    
#     return DataLoader(self.val, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

#   def test_dataloader(self):
#     return DataLoader(self.test, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)


###################
# Transformations # This can be hyperparameters as well - like try many of it
###################

train_transforms = transforms.Compose([
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

class H5Dataset(Dataset):
    def __init__(self, path):
        self.file_path = path
        self.dataset_x = None
        self.dataset_y = None
        ### Going to read the X part of the dataset - it's a different file
        with h5py.File(self.file_path + '_x.h5', 'r') as filex:
            self.dataset_x_len = len(filex['x'])

        ### Going to read the y part of the dataset - it's a different file
        with h5py.File(self.file_path + '_y.h5', 'r') as filey:
            self.dataset_y_len = len(filey['y'])

    def __getitem__(self, index):
        if self.dataset_x is None:
            self.dataset_x = h5py.File(self.file_path +'_x.h5', 'r')['x']
        if self.dataset_y is None:
            self.dataset_y = h5py.File(self.file_path +'_y.h5', 'r')['y']
        return (self.dataset_x[index], self.dataset_y[index])

    def __len__(self):
        assert self.dataset_x_len == self.dataset_y_len
        return self.dataset_x_len

In [ ]:
# train_datamodule = H5_PCAM_DataModule(drive_base_path + 'camelyonpatch_level_2_split_train')
# val_datamodule = H5_PCAM_DataModule(drive_base_path + 'camelyonpatch_level_2_split_valid')
# test_datamodule = H5_PCAM_DataModule(drive_base_path + 'camelyonpatch_level_2_split_test')

In [ ]:
# train_datamodule

In [ ]:
dataloader_params = {'batch_size': BATCH_SIZE, 'shuffle': True, 'num_workers': 2}

train_path = drive_base_path + 'camelyonpatch_level_2_split_train'
val_path = drive_base_path + 'camelyonpatch_level_2_split_valid'
test_path = drive_base_path + 'camelyonpatch_level_2_split_test'

test_dataset = H5Dataset(test_path)
test_loader = DataLoader(test_dataset, **dataloader_params)

val_dataset = H5Dataset(val_path)
dev_loader = DataLoader(val_dataset, **dataloader_params)

train_dataset = H5Dataset(train_path)
train_loader = DataLoader(train_dataset, **dataloader_params)

In [ ]:
for x, y in train_loader:
  print(x.shape)
  print(y.shape)
  break

torch.Size([16, 96, 96, 3])
torch.Size([16, 1, 1, 1])


In [ ]:
### Pytorch ligthning module
# class CNN_V2(nn.Module):

class CNN_V2(pl.LightningModule):
    """This is the CNN That the CS230 guys have used, taken from http://cs230.stanford.edu/projects_winter_2019/posters/15813053.pdf
       The model inheretes from nn.Module"""
    def __init__(self, p = 0.5):
        # log parameter
        wandb.config.dropout = p
        """The init method for initializaing the CNN model"""
        super(CNN_V2, self).__init__()
        # 1. Convolutional layers
        # Single image is in shape: 3x96x96 (CxHxW, H==W), RGB images
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 3, stride = 1, padding = 1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        
        self.dropout = nn.Dropout(p = p)
        
        # 2. FC layers to final output
        self.fc1 = nn.Linear(in_features = 128*6*6, out_features = 512)
        self.fc_bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(in_features = 512, out_features = 256)
        self.fc_bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(in_features = 256, out_features = 128)
        self.fc_bn3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(in_features = 128, out_features = 1)

        # log hyperparameters
        self.save_hyperparameters()

        self.train_acc = pl.metrics.Accuracy()
        self.valid_acc = pl.metrics.Accuracy()
        self.test_acc = pl.metrics.Accuracy()

    def forward(self, x):
        # Apply 4x...
        # Convolution Layers, followed by Batch Normalizations, Maxpool, and ReLU
        x = self.bn1(self.conv1(x))                      # batch_size x 96 x 96 x 16
        x = self.pool(F.relu(x))                         # batch_size x 48 x 48 x 16
        x = self.bn2(self.conv2(x))                      # batch_size x 48 x 48 x 32
        x = self.pool(F.relu(x))                         # batch_size x 24 x 24 x 32
        x = self.bn3(self.conv3(x))                      # batch_size x 24 x 24 x 64
        x = self.pool(F.relu(x))                         # batch_size x 12 x 12 x 64
        x = self.bn4(self.conv4(x))                      # batch_size x 12 x 12 x 128
        x = self.pool(F.relu(x))                         # batch_size x  6 x  6 x 128
        # Flatten the output for each image
        x = x.reshape(-1, self.num_flat_features(x))        # batch_size x 6*6*128
        
        # Apply 4 FC Layers
        x = self.fc1(x)
        x = self.fc_bn1(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        x = self.fc_bn2(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = self.fc3(x)
        x = self.fc_bn3(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = self.fc4(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [ ]:
####################
# Helper functions #
####################

def show_image(image, label):
    """Show image with label"""
    print('Label: ' + str(label[0]))
    plt.show(image)

def sigmoid(x):
    """This method calculates the sigmoid function"""
    return 1.0/(1.0 + np.exp(-x))

def training_accuracy(predicted, true, i, acc, tpr, tnr):
    """Taken from https://www.kaggle.com/krishanudb/cancer-detection-deep-learning-model-using-pytorch"""
    predicted = predicted.cpu() # Taking the predictions, why cpu and not device?
    true = true.cpu() # Taking the labels, why cpu and not device?
    
    predicted = (sigmoid(predicted.data.numpy()) > 0.5) # Using sigmoid above, if prediction > 0.5 it is 1
    true = true.data.numpy() # Numpy - can't combine above?
    accuracy = np.sum(predicted == true) / true.shape[0] # Accuracy is: (TP + TN)/(TP + TN + FN + FP)
    true_positive_rate = np.sum((predicted == 1) * (true == 1)) / np.sum(true == 1) # TPR: TP / (TP + FN) aka Recall
    true_negative_rate = np.sum((predicted == 0) * (true == 0)) / np.sum(true == 0) # TNR: TN / (FP + TN)
    acc = acc * (i) / (i + 1) + accuracy / (i + 1)
    tpr = tpr * (i) / (i + 1) + true_positive_rate / (i + 1)
    tnr = tnr * (i) / (i + 1) + true_negative_rate / (i + 1)
    return acc, tpr, tnr

def dev_accuracy(predicted, target):
    """Taken from https://www.kaggle.com/krishanudb/cancer-detection-deep-learning-model-using-pytorch"""
    predicted = predicted.cpu()
    target = target.cpu()
    predicted = (sigmoid(predicted.data.numpy()) > 0.5)
    true = target.data.numpy()
    accuracy = np.sum(predicted == true) / true.shape[0]
    true_positive_rate = np.sum((predicted == 1) * (true == 1)) / np.sum(true == 1)
    true_negative_rate = np.sum((predicted == 0) * (true == 0)) / np.sum(true == 0)
    return accuracy, true_positive_rate, true_negative_rate

def train_dev_test_split_indices(dataset_size, train_split=0.8, dev_split=0.1, seed=30):
    """Given a dataset size, returns the train/dev/test split indices"""
    indices = list(range(dataset_size))
    train_split_end = int(np.floor(train_split * dataset_size))
    dev_split_end = int(np.floor(dev_split * dataset_size)) + train_split_end
    np.random.seed(seed)
    np.random.shuffle(indices)
    train_indices = indices[:train_split_end]
    dev_indices = indices[train_split_end:dev_split_end]
    test_indices = indices[dev_split_end:]
    return train_indices, dev_indices, test_indices

def train_dev_test_split_indices_from_dataset(dataset, train_split=0.8, dev_split=0.1, seed=30):
    """Given a dataset, returns the train/dev/test split indices"""
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    train_split_end = int(np.floor(train_split * dataset_size))
    dev_split_end = int(np.floor(dev_split * dataset_size)) + train_split_end
    np.random.seed(seed)
    np.random.shuffle(indices)
    train_indices = indices[:train_split_end]
    dev_indices = indices[train_split_end:dev_split_end]
    test_indices = indices[dev_split_end:]
    return train_indices, dev_indices, test_indices

def fetch_state(epoch, model, optimizer, dev_loss_min, dev_acc_max):
    """Returns the state dictionary for a model and optimizer"""
    state = {
        'epoch': epoch,
        'dev_loss_min': dev_loss_min,
        'dev_acc_max': dev_acc_max,
        'state_dict': model.state_dict(),
        'optim_dict': optimizer.state_dict()
    }
    return state

def save_checkpoint(state, is_best = False, checkpoint = CHECKPOINT_DIR):
    """Taken from CS230 PyTorch Code Examples"""
    """Saves model and training parameters at checkpoint + 'last.pth.tar'. If is_best==True, also saves
    checkpoint + 'best.pth.tar'

    Args:
        state: (dict) contains model's state_dict, may contain other keys such as epoch, optimizer state_dict
        is_best: (bool) True if it is the best model seen till now
        checkpoint: (string) folder where parameters are to be saved
    """
    filepath = os.path.join(checkpoint, 'last_v2.pth.tar')
    if (not os.path.exists(checkpoint)):
        print("Checkpoint Directory does not exist! Making directory {}".format(checkpoint))
        os.mkdir(checkpoint)
    else:
        print("Checkpoint Directory exists! ")
    torch.save(state, filepath)
    if (is_best):
        shutil.copyfile(filepath, os.path.join(checkpoint, 'best_v2.pth.tar'))
        
def load_checkpoint(model, optimizer = None, checkpoint = CHECKPOINT_DIR):
    """Taken from CS230 PyTorch Code Examples"""
    """Loads model parameters (state_dict) from file_path. If optimizer is provided, loads state_dict of
    optimizer assuming it is present in checkpoint.

    Args:
        checkpoint: (string) filename which needs to be loaded
        model: (torch.nn.Module) model for which the parameters are loaded
        optimizer: (torch.optim) optional: resume optimizer from checkpoint
    """
    if not os.path.exists(checkpoint):
        print("File doesn't exist {}".format(checkpoint))
        checkpoint = None
        return
    checkpoint = torch.load(checkpoint)
    model.load_state_dict(checkpoint['state_dict'])

    if optimizer:
        optimizer.load_state_dict(checkpoint['optim_dict'])

    return checkpoint

In [ ]:
# Parameters
num_workers = 0
total_epochs = 0
num_epochs = 50
early_stop_limit = 10
bad_epoch_count = 0
stop = False
train_loss_min = np.Inf
dev_loss_min = np.Inf
dev_acc_max = 0

In [ ]:
# Load model
USE_GPU = torch.cuda.is_available()

model = CNN_V2()
if (USE_GPU):
    model.cuda()


# Hyperparameters
lr = 5e-4
wandb.config.learning_rate = lr

In [ ]:
# Optimizer + Loss Function
#optimizer = optim.Adam(model.parameters(), lr = lr)
optimizer = optim.SGD(model.parameters(), lr = lr)   # SWATS
criterion = nn.BCEWithLogitsLoss() # Binary Cross Entropy for binary classification - malignant / benign

# Load best checkpoint
best_checkpoint = os.path.join(CHECKPOINT_DIR, 'best_v2.pth.tar');
#checkpoint = load_checkpoint(model = model, optimizer = optimizer, checkpoint = best_checkpoint)
# checkpoint = load_checkpoint(model = model, optimizer = None, checkpoint = best_checkpoint) # SWATS
# total_epochs = None if checkpoint is None else checkpoint['epoch']
total_epochs = 0

# Initialize arrays for plot
train_loss_arr = []
train_acc_arr = []
train_tpr_arr = []
train_tnr_arr = []

dev_loss_arr = []
dev_acc_arr = []
dev_tpr_arr = []
dev_tnr_arr = []

In [ ]:
# Loop over the dataset multiple times
total_num_epochs = total_epochs + num_epochs
for epoch in range(num_epochs):
    curr_epoch = total_epochs + epoch + 1
    # Keep track of training loss
    train_loss = []
    # Keep track of dev loss
    dev_loss = []
    # Keep track of accuracy measurements
    acc, tpr, tnr = 0.0, 0.0, 0.0

    # Train the model
    start_time = time.time()
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        if USE_GPU:
            data, target = image.cuda(), label.cuda()
        else:
            data, target = image, label
        # Zero the parameter gradients
        optimizer.zero_grad()
        # Forward pass
        # print("Before re-shape", data.shape)
        data = data.permute(0, 3, 1, 2).float() # these steps can be done in a separate class e.g. ToTensor from https://www.kaggle.com/krishanudb/cancer-detection-deep-learning-model-using-pytorch
        # print("After re-shape", data.shape)
        output = model(data)
        # Update target to be the same dimensions as output
        target = target.view(output.shape[0], 1).float()
        # Get accuracy measurements
        acc, tpr, tnr = training_accuracy(output, target, batch_idx, acc, tpr, tnr)
        # Calculate the batch's loss
        curr_train_loss = criterion(output, target)
        # Update the training loss
        train_loss.append(curr_train_loss.item())
        # Backward pass
        curr_train_loss.backward()
        # Perform a single optimization step to update parameters
        optimizer.step()
        # Print debug info every 64 batches
        if (batch_idx) % 64 == 0:
            print('Epoch {}/{}; Iter {}/{}; Loss: {:.4f}; Acc: {:.3f}; True Pos: {:.3f}; True Neg: {:.3f}'
                   .format(curr_epoch, total_num_epochs, batch_idx + 1, len(train_loader), curr_train_loss.item(), acc, tpr, tnr))
            
    end_time = time.time()
    
    # Evaluate the model
    model.eval()
    with torch.no_grad():
        for batch_idx, (image, label) in enumerate(dev_loader):
            if USE_GPU:
                data, target = image.cuda(), label.cuda()
            else:
                data, target = image, label
            # Get predicted output
            # .permute(0, 3, 1, 2)
            # print("Before re-shape", data.shape)
            data = data.permute(0, 3, 1, 2).float()
            # print("After re-shape", data.shape)
            output = model(data)
            # Update target to be the same dimensions as output
            target = target.view(output.shape[0], 1).float()
            # Get accuracy measurements
            dev_acc, dev_tpr, dev_tnr = dev_accuracy(output, target)
            # Calculate the batch's loss
            curr_dev_loss = criterion(output, target)
            # Update the dev loss
            dev_loss.append(curr_dev_loss.item())
    
    # Calculate average loss
    avg_train_loss = np.mean(np.array(train_loss))
    avg_dev_loss = np.mean(np.array(dev_loss))
    
    # Update dev loss arrays
    dev_loss_arr.append(avg_dev_loss)
    dev_acc_arr.append(dev_acc)
    dev_tpr_arr.append(dev_tpr)
    dev_tnr_arr.append(dev_tnr)

    # Update training loss arrays
    train_loss_arr.append(avg_train_loss)
    train_acc_arr.append(acc)
    train_tpr_arr.append(tpr)
    train_tnr_arr.append(tnr)

    print('Epoch {}/{}; Avg. Train Loss: {:.4f}; Train Acc: {:.3f}; Train TPR: {:.3f}; Train TNR: {:.3f}; Epoch Time: {} mins; \nAvg. Dev Loss: {:.4f}; Dev Acc: {:.3f}; Dev TPR: {:.3f}; Dev TNR: {:.3f}\n'
        .format(curr_epoch, total_num_epochs, avg_train_loss, acc, tpr, tnr, round((end_time - start_time)/ 60., 2), avg_dev_loss, dev_acc, dev_tpr, dev_tnr))
    
    wandb.log({'epoch': curr_epoch, 'loss': avg_train_loss, 'accuracy': acc, 'tpr': tpr, 'time_per_epoch_min': round((end_time - start_time)/ 60., 2)})

    if avg_dev_loss < dev_loss_min:
        print('Dev loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
              .format(dev_loss_min, avg_dev_loss))
        dev_loss_min = avg_dev_loss
        is_best = False
        if (dev_acc >= dev_acc_max):
            is_best = True
            dev_acc_max = dev_acc
        state = fetch_state(epoch = curr_epoch, model = model, optimizer = optimizer, 
                            dev_loss_min = dev_loss_min, 
                            dev_acc_max = dev_acc_max)
        save_checkpoint(state = state, is_best = is_best)
        bad_epoch_count = 0
    # If dev loss didn't improve, increase bad_epoch_count and stop if
    # bad_epoch_count >= early_stop_limit
    else:
        bad_epoch_count += 1
        print('{} epochs of increasing dev loss ({:.6f} --> {:.6f}).'
              .format(bad_epoch_count, dev_loss_min, avg_dev_loss))
        if (bad_epoch_count >= early_stop_limit):
            print('Stopping training')
            stop = True

    if (stop):
        break

Epoch 1/50; Iter 1/16384; Loss: 0.7694; Acc: 0.500; True Pos: 0.333; True Neg: 0.714
Epoch 1/50; Iter 65/16384; Loss: 0.7352; Acc: 0.514; True Pos: 0.316; True Neg: 0.724
Epoch 1/50; Iter 129/16384; Loss: 0.6738; Acc: 0.534; True Pos: 0.342; True Neg: 0.738
Epoch 1/50; Iter 193/16384; Loss: 0.7965; Acc: 0.548; True Pos: 0.372; True Neg: 0.734
Epoch 1/50; Iter 257/16384; Loss: 0.5914; Acc: 0.561; True Pos: 0.398; True Neg: 0.733
Epoch 1/50; Iter 321/16384; Loss: 0.7078; Acc: 0.572; True Pos: 0.418; True Neg: 0.737
Epoch 1/50; Iter 385/16384; Loss: 0.6352; Acc: 0.582; True Pos: 0.443; True Neg: 0.735
Epoch 1/50; Iter 449/16384; Loss: 0.5575; Acc: 0.588; True Pos: 0.458; True Neg: 0.733
Epoch 1/50; Iter 513/16384; Loss: 0.6130; Acc: 0.593; True Pos: 0.473; True Neg: 0.728
Epoch 1/50; Iter 577/16384; Loss: 0.5980; Acc: 0.597; True Pos: 0.487; True Neg: 0.723
Epoch 1/50; Iter 641/16384; Loss: 0.6307; Acc: 0.603; True Pos: 0.498; True Neg: 0.723
Epoch 1/50; Iter 705/16384; Loss: 0.5781; Acc:

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in long_scalars


Epoch 2/50; Iter 10753/16384; Loss: 0.2802; Acc: 0.826; True Pos: 0.844; True Neg: nan
Epoch 2/50; Iter 10817/16384; Loss: 0.2793; Acc: 0.826; True Pos: 0.844; True Neg: nan
Epoch 2/50; Iter 10881/16384; Loss: 0.4042; Acc: 0.826; True Pos: 0.844; True Neg: nan
Epoch 2/50; Iter 10945/16384; Loss: 0.6075; Acc: 0.826; True Pos: 0.844; True Neg: nan
Epoch 2/50; Iter 11009/16384; Loss: 0.3055; Acc: 0.826; True Pos: 0.844; True Neg: nan
Epoch 2/50; Iter 11073/16384; Loss: 0.4908; Acc: 0.826; True Pos: 0.844; True Neg: nan
Epoch 2/50; Iter 11137/16384; Loss: 0.2076; Acc: 0.826; True Pos: 0.844; True Neg: nan
Epoch 2/50; Iter 11201/16384; Loss: 0.9127; Acc: 0.826; True Pos: 0.844; True Neg: nan
Epoch 2/50; Iter 11265/16384; Loss: 0.3751; Acc: 0.826; True Pos: 0.844; True Neg: nan
Epoch 2/50; Iter 11329/16384; Loss: 0.3236; Acc: 0.826; True Pos: 0.844; True Neg: nan
Epoch 2/50; Iter 11393/16384; Loss: 0.1906; Acc: 0.826; True Pos: 0.844; True Neg: nan
Epoch 2/50; Iter 11457/16384; Loss: 0.5750;